In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jd_comment_with_label/jd_comment_data.xlsx


In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import os

2025-05-21 16:07:57.638409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747843677.821579      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747843677.877322      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# 加载google的bert中文模型
from transformers import AutoModel

bert = AutoModel.from_pretrained('google-bert/bert-base-chinese')

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

In [4]:
# 加载预训练模型配置参数
from transformers import AutoConfig, BertModel

config = AutoConfig.from_pretrained('google-bert/bert-base-chinese')

# 添加自定义参数传给模型构建方法
# config = AutoConfig.from_pretrained('google-bert/bert-base-chinese',foo=1,bar=True)

print(config)


BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.51.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}



In [5]:
import pandas as pd
# 替换为你的数据集路径
data_path = '/kaggle/input/jd_comment_with_label/jd_comment_data.xlsx'

# 加载评论数据
df = pd.read_excel(data_path)
# 查看实际列名
print(df.columns.tolist())
df.head()



['爬取时间(__time)', '爬取链接(__url)', '商品ID(product_id)', '评价时间(publish_time)', '评分（总分5分）(score)', '评价内容(content)', '评价者(author_name)', '评价者会员等级(author_level)', '商品sku(product_sku)', '评价标签(tags)']


,爬取时间(__time),爬取链接(__url),商品ID(product_id),评价时间(publish_time),评分（总分5分）(score),评价内容(content),评价者(author_name),评价者会员等级(author_level),商品sku(product_sku),评价标签(tags)
0,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550631798,5,此用户未填写评价内容,j***e,注册会员,单机版 小D黑色,[]
1,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633151,5,此用户未填写评价内容,c***n,钻石会员,单机版 小D黑色,[]
2,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633330,3,此用户未填写评价内容,j***1,银牌会员,单机版 小D黑色,[]
3,2019-03-08 00:50:34,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633401,5,此用户未填写评价内容,苗***4,钻石会员,单机版 小D黑色,[]
4,2019-03-08 00:50:33,https://sclub.jd.com/comment/productPageCommen...,4722324,1550633461,5,此用户未填写评价内容,J***3,注册会员,单机版 小D黑色,[]


In [6]:
# 提取“评价内容”和“评分”作为输入和标签
df = df[['评价内容(content)', '评分（总分5分）(score)']].dropna()

# 重命名列，便于后续统一处理
df.columns = ['content', 'label']

# 显示前几行确认
print(df.head())
# 显示标签分布（1~5分）
print(df['label'].value_counts())

      content  label
0  此用户未填写评价内容      5
1  此用户未填写评价内容      5
2  此用户未填写评价内容      3
3  此用户未填写评价内容      5
4  此用户未填写评价内容      5
label
5    68572
4     1162
1     1096
3      679
2      309
Name: count, dtype: int64


In [7]:
from sklearn.model_selection import train_test_split
# 转换为 int 并划分训练验证集
df['label'] = df['label'].astype(int)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['content'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42)

In [8]:
class JDDataset(Dataset):
    def __init__(self, texts, labels):
        self.data = list(zip(texts, labels))

    def __getitem__(self, idx):
        return self.data[idx][0], self.data[idx][1] - 1  # 保证是 tuple 类型 (str, int)

    def __len__(self):
        return len(self.data)


def build_collate(tokenizer):
    def collate_fn(batch):
        # 正确：batch 是 List[Tuple[str, int]]
        sentences, labels = zip(*batch)
        model_inputs = tokenizer(
            list(sentences), return_tensors='pt', padding=True, truncation=True, max_length=128
        )
        labels = torch.tensor(labels)
        return model_inputs, labels
    return collate_fn


tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

train_dataset = JDDataset(train_texts, train_labels)
val_dataset = JDDataset(val_texts, val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=build_collate(tokenizer))
val_loader = DataLoader(val_dataset, batch_size=32, collate_fn=build_collate(tokenizer))


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [9]:
def get_model(freeze_bert=False):
    model = AutoModelForSequenceClassification.from_pretrained(
        'google-bert/bert-base-chinese', num_labels=5)
    if freeze_bert:
        for param in model.bert.parameters():
            param.requires_grad = False
    return model

In [10]:
def train(model, freeze_bert=False):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=2e-5)
    total_steps = len(train_loader) * 3
    scheduler = get_scheduler("linear", optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    writer = SummaryWriter(log_dir=f'./runs/freeze_{freeze_bert}')
    global_step = 0

    for epoch in range(3):
        model.train()
        total_loss, total_correct, total = 0, 0, 0

        for model_inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch} Train"):
            model_inputs = {k: v.to(device) for k, v in model_inputs.items()}
            labels = labels.to(device)

            outputs = model(**model_inputs, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            total_correct += (preds == labels).sum().item()
            total += labels.size(0)

            writer.add_scalar("Loss/train_step", loss.item(), global_step)
            global_step += 1

        writer.add_scalar("Accuracy/train", total_correct / total, epoch)

        # 验证集
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for model_inputs, labels in val_loader:
                model_inputs = {k: v.to(device) for k, v in model_inputs.items()}
                labels = labels.to(device)
                outputs = model(**model_inputs)
                preds = torch.argmax(outputs.logits, dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        writer.add_scalar("Accuracy/val", val_correct / val_total, epoch)

    writer.close()
    torch.save(model.state_dict(), f'/kaggle/working/bert_freeze_{freeze_bert}.pt')
    print(f"模型已保存: /kaggle/working/bert_freeze_{freeze_bert}.pt")

In [ ]:
model1 = get_model(freeze_bert=True)
train(model1, freeze_bert=True)

model2 = get_model(freeze_bert=False)
train(model2, freeze_bert=False)

In [ ]:
def predict(text_list, model_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForSequenceClassification.from_pretrained(
        'google-bert/bert-base-chinese', num_labels=5)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    inputs = tokenizer(text_list, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=1)
    return preds.cpu().numpy() + 1  # 转换为原始1~5分

# 示例
example = ["手机拍照效果真棒啊，把人拍成马赛克了"]
print("预测评分:", predict(example, '/kaggle/working/bert_freeze_True.pt'))
